In [1]:
#!/usr/bin/env python3


# import necessary modules
import smtplib
import pandas as pd
import os
import glob
import sys

from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

In [2]:
# set home directory so can be used on all OS
home = os.path.expanduser('~')

In [3]:
# Find most recently exported files from registry and repository
# reg_file should be report XX
# repo_file should be report XX
# link_file should be downloaded from XX as according to wiki
reg_file = max(glob.iglob(home+'/Downloads/RDRPRegistry_DATA_*'), key=os.path.getctime)
repo_file = max(glob.iglob(home+'/Downloads/RDRPRepository_DATA_*'), key=os.path.getctime)
link_file = max(glob.iglob(home+'/Downloads/RDRPRepository_Participants_*'), key=os.path.getctime)

In [4]:
print(reg_file)
print(repo_file)
print(link_file)

/home/pdonnelly/Downloads/RDRPRegistry_DATA_2018-09-12_1336.csv
/home/pdonnelly/Downloads/RDRPRepository_DATA_2018-09-12_1337.csv
/home/pdonnelly/Downloads/RDRPRepository_Participants_2018-09-12_1337.csv


In [5]:
# Load data from each using record_id as index
reg_data = pd.read_csv(reg_file, index_col='record_id')
repo_data = pd.read_csv(repo_file, index_col='record_id')
link_data = pd.read_csv(link_file, skiprows=1, header=None, names=['dummy', 'na', 
    'record_id', 'd', 'e', 'f', 'survey', 'link'], index_col='record_id', dtype=object) # headers in the file are poorly written. na, d, e, f are all not used 



In [27]:
# Extract links for survey queues via innerjoin
reg_data = reg_data.join(link_data)


In [29]:
# Use the repo_data to select which individuals will receive emails
subs = pd.DataFrame(reg_data.ix[repo_data.index])

In [31]:
# Check to make sure we are sending surveys
# Script is set up to take any text formatted according to rules published on wiki
if input('Would you like to send out the survey queue? y/n ') == 'y':
    form = open(home+'/git/redcap/email_temp/survey.txt')
    form = form.read()
    sys.stdout.write("Survey script loaded\n")
else:
    sys.stdout.write('Select one of the following templates:\n')
    os.system('ls '+home+'/git/redcap/email_temp')
    selection = input('Which of the above templates would you like to load? ')
    form = open(home+'/git/redcap/email_temp/'+selection)
    form = form.read()
    sys.stdout.write(selection+' loaded')

Would you like to send out the survey queue? y/n y
Survey script loaded


In [42]:
# Check to see what subject line should be used, then set
subl_sel = input('Please select a subject line for your message:\n(1) Please complete the attached surveys\n(2) Greetings from the UW Reading & Dyslexia Research Program\n(3) New study opportunity with the UW Reading & Dyslexia Research Program\n(4) Custom\nPlease enter the number of your choice: ')
if subl_sel == '1':
    subl = 'Please complete the attached surveys'
elif subl_sel == '2':
    subl = 'Greetings from the UW Reading & Dyslexia Research Program'
elif subl_sel == '3':
    subl = 'New study opportunity UW Reading & Dyslexia Research Program'
else:
    subl = input('Please enter text for email Subject line: ')

Please select a subject line for your message:
(1) Please complete the attached surveys
(2) Greetings from the UW Reading & Dyslexia Research Program
(3) New study opportunity with the UW Reading & Dyslexia Research Program
(4) Custom
Please enter the number of your choice: 1


In [ ]:
# Set all features not to be iterated
rdrp_email = "rdrp@uw.edu"

body = """\
<html>
  <head></head>
  <body>
    <p>{}
    </p>
  </body>
</html>
"""

In [ ]:
server = smtplib.SMTP('smtp.gmail.com', 587)
server.starttls()
server.login("rdrp@uw.edu", input('Please enter password for rdrp@uw.edu: '))

In [53]:
subs.index[pd.notnull(subs['email'])]

Int64Index([   5,    9,   10,   18,   28,   29,   30,   35,   36,   45,
            ...
            1092, 1095, 1098, 1101, 1102, 1103, 1108, 1110, 1112, 1116],
           dtype='int64', name='record_id', length=263)

In [ ]:
# Scrape subject data from info, input into form.format in the same order as the positions
# to be filled in.
for sub in subs.index[pd.notnull(subs['email'])]: #excludes subs without email address defined
    if pd.isnull(reg_data.parent_first_name[sub]):
        ename = subs.first_name[sub]
        link = subs.link[sub]
        your = 'your'
        body = form.format(ename=ename, link=link, your=your)
        sub_email = subs.email[sub]
        # Create message container - the correct MIME type is multipart/alternative.
        msg = MIMEMultipart()
        msg['Subject'] = subl
        msg['From'] = rdrp_email
        msg['To'] = sub_email
        
        # Create the body of the message.
        html = body.format(form)
        
        # Record the MIME types of text/html and attach into message container.
        msg.attach(MIMEText(html, 'html'))
        
        # sendmail function takes 3 arguments: sender's address, recipient's address
        # and message to send - here it is sent as one string.
        #server.sendmail(rdrp_email, sub_email, msg.as_string())
        
        # update on status
        sys.stdout.write('Email sent to {}\n'.format(repo_data.sid.ix[sub]))
    else:
        ename = subs.parent_first_name[sub]
        link = subs.link[sub]
        your = subs.first_name[sub] + "'s"
        body = form.format(ename=ename, link=link, your=your, unsubscribe=subs.survey[sub], lab_name=lab_name, lab_role=lab_role)
        sub_email = subs.email[sub]
        # Create message container - the correct MIME type is multipart/alternative.
        msg = MIMEMultipart()
        msg['Subject'] = subl
        msg['From'] = rdrp_email
        msg['To'] = sub_email
        
        # Create the body of the message.
        html = body.format(form)
        
        # Record the MIME types of text/html and attach into message container.
        msg.attach(MIMEText(html, 'html'))
        
        # sendmail function takes 3 arguments: sender's address, recipient's address
        # and message to send - here it is sent as one string.
        #server.sendmail(rdrp_email, sub_email, msg.as_string())
        
        # update on status
        sys.stdout.write('Email sent to {}\n'.format(repo_data.sid.ix[sub]))
       
#close server
server.quit()

In [ ]:
# Write out all subjects that have missing email data in reg
no_email = repo_data[pd.isnull(subs['email'])]
sys.stdout.write('No email in reg for subjects:\n')
for sub in no_email.index:
    sys.stdout.write('{}\n'.format(no_email.sid[sub]))

In [ ]:
'{}'.format(repo_data.sid[pd.isnull(subs['email'])])

In [ ]:
subs.index[pd.notnull(subs['email'])]